## Ansible Config

Search order

1. File specified by the ANSIBLE_CONFIG environment variable
2. ./ansible.cfg (ansible.cfg in the current directory)
3. ~/.ansible.cfg (.ansible.cfg in your home directory)
4. /etc/ansible/ansible.cfg

Some specific configuration, just put it either in ansible.cfg file in local directory or in home directory



## Ping a machine


    ansible <machine> -i <hosts file> -m <module>

this is an example hosts file with basic configuration.

    ansible ansible_host=ansible ansible_user=ansible ansible_private_key_file=/root/id_rsa
    debian  ansible_host=debian  ansible_user=ansible ansible_private_key_file=/root/id_rsa

`ansible` and `debian` are the machine names. 

`ansible` is the machine which installed ansible. It can also be provisioned by ansible itself

In [5]:
!ansible ansible -i 0-basic/hosts -m ping

ansible | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


In [6]:
!ansible debian  -i 0-basic/hosts -m ping

debian | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


## Execute a specific command

    ansible localuser -i playbooks/hosts -m command -a uptime
    
The `command` module is so commonly used so that we can omit it

    ansible localuser -i playbooks/hosts -a uptime
    
We can quote command parameters with '' or " " if it contains space

    ansible localuser -i playbooks/hosts -a 'ip addr'
    ansible localuser -i playbooks/hosts -a "ip addr"   

In [8]:
!ansible debian  -i 0-basic/hosts -m command -a uptime

debian | SUCCESS | rc=0 >>
 04:19:52 up 25 min,  1 user,  load average: 0.09, 0.02, 0.01



The command module is so commonly used so that we can omit it

In [9]:
!ansible debian  -i 0-basic/hosts -a uptime

debian | SUCCESS | rc=0 >>
 04:20:07 up 25 min,  1 user,  load average: 0.08, 0.02, 0.00



We can quote command parameters with '' or " " if it contains space

In [15]:
!ansible debian -i 0-basic/hosts -a "ip addr" 

debian | SUCCESS | rc=0 >>
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
2: tunl0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1
    link/ipip 0.0.0.0 brd 0.0.0.0
3: ip6tnl0@NONE: <NOARP> mtu 1452 qdisc noop state DOWN group default qlen 1
    link/tunnel6 :: brd ::
9: eth0@if10: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:12:00:03 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.18.0.3/16 brd 172.18.255.255 scope global eth0
       valid_lft forever preferred_lft forever



## Execute with root

the option -b is used to execute command in root
    
Prerequisite

- sudo command installed
- the user has sudo permission. Add line like this by typing command visudo "user   ALL=(ALL) NOPASSWD: ALL"



In [12]:
!ansible debian -i 0-basic/hosts -b -a "cat /etc/shadow"

debian | SUCCESS | rc=0 >>
root:$6$uBtdcILg$.wU8ALJj.KhAISAf9tWXFHwQ03EciyY8dFxVDUhYOtBJBUP13P6T4e7gvX4u5kFZNDI4uJdUAMoNS3zBrjK47/:17765:0:99999:7:::
daemon:*:17707:0:99999:7:::
bin:*:17707:0:99999:7:::
sys:*:17707:0:99999:7:::
sync:*:17707:0:99999:7:::
games:*:17707:0:99999:7:::
man:*:17707:0:99999:7:::
lp:*:17707:0:99999:7:::
mail:*:17707:0:99999:7:::
news:*:17707:0:99999:7:::
uucp:*:17707:0:99999:7:::
proxy:*:17707:0:99999:7:::
www-data:*:17707:0:99999:7:::
backup:*:17707:0:99999:7:::
list:*:17707:0:99999:7:::
irc:*:17707:0:99999:7:::
gnats:*:17707:0:99999:7:::
nobody:*:17707:0:99999:7:::
_apt:*:17707:0:99999:7:::
messagebus:*:17764:0:99999:7:::
sshd:*:17764:0:99999:7:::
systemd-timesync:*:17764:0:99999:7:::
systemd-network:*:17764:0:99999:7:::
systemd-resolve:*:17764:0:99999:7:::
systemd-bus-proxy:*:17764:0:99999:7:::
ansible:ansible:17765:0:99999:7:::
test:test:17765:0:99999:7:::



## Execute command on multiple machines

by define a group of machine in hosts file. we can send commands to all of them.

basic is a group of machine, which as ansible and debian

    ansible ansible_host=ansible ansible_user=ansible ansible_private_key_file=/root/id_rsa
    debian  ansible_host=debian  ansible_user=ansible ansible_private_key_file=/root/id_rsa

    [basic]
    ansible
    debian

In [18]:
!ansible basic -i 0-basic/hosts -a "ip addr show eth0" 

ansible | SUCCESS | rc=0 >>
7: eth0@if8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:12:00:02 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.18.0.2/16 brd 172.18.255.255 scope global eth0
       valid_lft forever preferred_lft forever

debian | SUCCESS | rc=0 >>
9: eth0@if10: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:12:00:03 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.18.0.3/16 brd 172.18.255.255 scope global eth0
       valid_lft forever preferred_lft forever



`command` module does not support variables like `$HOME` and operations like `"<"`, `">"`, `"|"`, `";"` and `"&"` will not work (use the shell module if you need these features).

In [19]:
!ansible basic -i 0-basic/hosts -m shell -a "ip addr | grep inet" 

ansible | SUCCESS | rc=0 >>
    inet 127.0.0.1/8 scope host lo
    inet 172.18.0.2/16 brd 172.18.255.255 scope global eth0

debian | SUCCESS | rc=0 >>
    inet 127.0.0.1/8 scope host lo
    inet 172.18.0.3/16 brd 172.18.255.255 scope global eth0



## First playbook

In [20]:
!ansible-playbook -i 0-basic/hosts 0-basic/first-playbook.yml


PLAY [first playbook] **********************************************************

TASK [setup] *******************************************************************
ok: [ansible]
ok: [debian]

TASK [execute a command] *******************************************************
changed: [ansible]
changed: [debian]

PLAY RECAP *********************************************************************
ansible                    : ok=2    changed=1    unreachable=0    failed=0   
debian                     : ok=2    changed=1    unreachable=0    failed=0   

